In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession\
                    .builder\
                    .master("spark://spark-master:7077")\
                    .appName("Day_6_DataFrame_2")\
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/02 10:20:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [4]:
#df1.join(df2,['col1','col2'],'inner')

employee_schema = StructType([
                            StructField("emp_id", IntegerType(), True),
                            StructField("emp_name", StringType(), True),
                            StructField("dept_id", IntegerType(), True),
                            StructField("salary", IntegerType(), True),
                            StructField("manager_id", IntegerType(), True),
                            StructField("emp_age", IntegerType(), True)
                            ])

employee_data = [
(1, "Ankit", 100, 10000, 4, 39),
(2, "Mohit", 100, 15000, 5, 48),
(3, "Vikas", 100, 10000, 4, 37),
(4, "Rohit", 100, 5000, 2, 16),
(5, "Mudit", 200, 12000, 6, 55),
(6, "Agam", 200, 12000, 2, 14),
(7, "Sanjay", 200, 9000, 2, 13),
(8, "Ashish", 200, 5000, 2, 12),
(9, "Mukesh", 300, 6000, 6, 51),
(10, "Rakesh", 500, 7000, 6, 50)
]
employee_df = spark.createDataFrame(employee_data, employee_schema)

dept_schema = StructType([
StructField("dept_id", IntegerType(), True),
StructField("dept_name", StringType(), True)
])
dept_data = [
(100, "Analytics"),
(200, "IT"),
(300, "HR"),
(400, "Text Analytics")
]
dept_df = spark.createDataFrame(dept_data, dept_schema)

employee_df.show()
dept_df.show()

+------+--------+-------+------+----------+-------+
|emp_id|emp_name|dept_id|salary|manager_id|emp_age|
+------+--------+-------+------+----------+-------+
|     1|   Ankit|    100| 10000|         4|     39|
|     2|   Mohit|    100| 15000|         5|     48|
|     3|   Vikas|    100| 10000|         4|     37|
|     4|   Rohit|    100|  5000|         2|     16|
|     5|   Mudit|    200| 12000|         6|     55|
|     6|    Agam|    200| 12000|         2|     14|
|     7|  Sanjay|    200|  9000|         2|     13|
|     8|  Ashish|    200|  5000|         2|     12|
|     9|  Mukesh|    300|  6000|         6|     51|
|    10|  Rakesh|    500|  7000|         6|     50|
+------+--------+-------+------+----------+-------+

+-------+--------------+
|dept_id|     dept_name|
+-------+--------------+
|    100|     Analytics|
|    200|            IT|
|    300|            HR|
|    400|Text Analytics|
+-------+--------------+



In [6]:
employee_df.groupBy(col('dept_id')).agg(sum('salary')).orderBy(sum(col('salary')).desc()).show()

[Stage 15:========================================>             (151 + 4) / 200]

+-------+-----------+
|dept_id|sum(salary)|
+-------+-----------+
|    100|      40000|
|    200|      38000|
|    500|       7000|
|    300|       6000|
+-------+-----------+



In [7]:
window_spec = Window.partitionBy('dept_id')
window_spec2=Window.orderBy(col('total_salary_dept').desc())

employee_df.withColumn('total_salary_dept',sum('salary').over(window_spec))\
            .withColumn('dense_rank',dense_rank().over(window_spec2))\
            .orderBy(col('dense_rank')).show()

26/02/02 10:32:01 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
[Stage 17:=============================================>        (167 + 4) / 200]

+------+--------+-------+------+----------+-------+-----------------+----------+
|emp_id|emp_name|dept_id|salary|manager_id|emp_age|total_salary_dept|dense_rank|
+------+--------+-------+------+----------+-------+-----------------+----------+
|     1|   Ankit|    100| 10000|         4|     39|            40000|         1|
|     2|   Mohit|    100| 15000|         5|     48|            40000|         1|
|     3|   Vikas|    100| 10000|         4|     37|            40000|         1|
|     4|   Rohit|    100|  5000|         2|     16|            40000|         1|
|     5|   Mudit|    200| 12000|         6|     55|            38000|         2|
|     6|    Agam|    200| 12000|         2|     14|            38000|         2|
|     7|  Sanjay|    200|  9000|         2|     13|            38000|         2|
|     8|  Ashish|    200|  5000|         2|     12|            38000|         2|
|    10|  Rakesh|    500|  7000|         6|     50|             7000|         3|
|     9|  Mukesh|    300|  6

In [8]:
spark.stop()